In [ ]:
pip install openai

In [ ]:

import json
def read_json_file(file_name):
    dataset = []
    with open(file_name, 'r') as file:
        for l in file:
            onj = json.loads(l.strip())
            dataset.append(onj)
    return dataset
data = read_json_file('/content/Mental_Health_Dataset.jsonl')

In [ ]:
import json
dataset = []

for tags in data:
    for pattern in tags["patterns"]:
      for response in tags["responses"]:
        dataset.append({"messages":[{"role": "user", "content": pattern},{"role":"assistant","content":response}]})

with open('gptformat.jsonl', 'w') as f:
    for obj in dataset:
        f.write(json.dumps(obj) + '\n')


In [ ]:
len(dataset)

661

In [ ]:
from openai import OpenAI

In [ ]:
#API key is hidden for a reason
client = OpenAI(api_key="Your_API_KEY")

In [ ]:
client.files.create(file=open("/content/gptformat.jsonl", "rb"), purpose="fine-tune")

FileObject(id='file-3znuEjd0RLvRLutofkRiOlL2', bytes=124138, created_at=1712117254, filename='gptformat.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
client.fine_tuning.jobs.list(limit=10)


SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-XwEPqDWJF8S6yW7WzOQaNUvA', created_at=1712114020, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Sna937AUwyVn16L9sNuPdrDT', result_files=[], status='cancelled', trained_tokens=None, training_file='file-NBqCqiU2R4lWSOZWwpDnde8j', validation_file=None, user_provided_suffix=None, seed=846812460), FineTuningJob(id='ftjob-tZNhHApV3AYqWUxEMvcPXBz1', created_at=1712113032, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-Sna937AUwyVn16L9sNuPdrDT', result_files=[], status='cancelled', trained_tokens=None, tra

In [ ]:
# client.fine_tuning.jobs.create(
#     model="gpt-3.5-turbo",
#     training_file="file-NBqCqiU2R4lWSOZWwpDnde8j",
# )

In [ ]:
completion = client.chat.completions.create(model="ft:gpt-3.5-turbo-0125:personal::99mAWS07", messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is anxiety"},
])

chat = completion.choices[0].message
print(chat)

ChatCompletionMessage(content='Anxiety is a feeling of worry, nervousness, or unease about something with an uncertain outcome.', role='assistant', function_call=None, tool_calls=None)


In [ ]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.4/313.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl size=5584 sha256=ae90c5b280102648002a4ff36cd7ede98fe9bf912b5d2016d11369136300fd73
  Stored in directory: /root/.cache/pip/wheels/bd/65/9a/671fc6dcde07d4418df0c592f8df512b26d7a0029c2a23dd81
Successfully built ffmpy


In [ ]:
import gradio as g

In [ ]:
def generate_response(text):
  completion = client.chat.completions.create(model="ft:gpt-3.5-turbo-0125:personal::99mAWS07", messages=[
      {"role": "system", "content": "You are a helpful assistant."},
      {"role": "user", "content": text},
    ])
  return completion.choices[0].message.content.strip()

def chatbot(input, prev):
    if(prev ==None):
      prev = []
    prompt = " "
    for text in prev:
      for data in text:
        prompt= ' '.join([data] + [input])
    output = generate_response(prompt)
    prev.append((input, output))
    return prev, prev


gradio_interface = g.Blocks()


with gradio_interface:
    g.Markdown("""<h1>GPT turbo Mental Health Chatbot</h1>
    """)
    chats = g.Chatbot()
    text = g.Textbox(placeholder="")
    chatbot_state = g.State()
    button= g.Button("SEND")
    button.click(chatbot, inputs=[text,chatbot_state], outputs=[chats,chatbot_state])

gradio_interface.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://52916e454fb86bb10c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
